In [1]:
# Import libraries
from paretoset import paretoset
import numpy as np
import pandas as pd
import geopandas as gpd
import momepy
import time
import copy
import networkx as nx

In [2]:
import networkx as nx

# Create a 5x5 grid graph
G = nx.Graph()

# Add nodes to the graph
for i in range(5):
    for j in range(5):
        G.add_node((i, j))

# Add edges to form the grid structure
for i in range(4):
    for j in range(5):
        G.add_edge((i, j), (i + 1, j))
        G.add_edge((i, j), (i, j + 1))

# Assign costs for length and number of crossings
length_costs = [
    [1, 2, 3, 4, 5],
    [2, 3, 4, 5, 6],
    [3, 4, 5, 6, 7],
    [4, 5, 6, 7, 8],
    [5, 6, 7, 8, 9]
]

crossings_costs = [
    [1, 1, 1, 1, 1],
    [1, 2, 2, 2, 1],
    [1, 2, 3, 2, 1],
    [1, 2, 2, 2, 1],
    [1, 1, 1, 1, 1]
]

# Assign the costs for each edge
for u, v in G.edges():
    u_x, u_y = u
    v_x, v_y = v
    length_cost = length_costs[u_x][u_y]
    crossings_cost = crossings_costs[u_x][u_y]
    G[u][v]['length'] = length_cost
    G[u][v]['crossings'] = crossings_cost

# Print the graph information
print("Graph nodes:", G.nodes())
print("Graph edges:", G.edges(data=True))
S = (0, 0)
T = (3, 5)


Graph nodes: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (0, 5), (1, 5), (2, 5), (3, 5)]
Graph edges: [((0, 0), (1, 0), {'length': 1, 'crossings': 1}), ((0, 0), (0, 1), {'length': 1, 'crossings': 1}), ((0, 1), (1, 1), {'length': 2, 'crossings': 1}), ((0, 1), (0, 2), {'length': 2, 'crossings': 1}), ((0, 2), (1, 2), {'length': 3, 'crossings': 1}), ((0, 2), (0, 3), {'length': 3, 'crossings': 1}), ((0, 3), (1, 3), {'length': 4, 'crossings': 1}), ((0, 3), (0, 4), {'length': 4, 'crossings': 1}), ((0, 4), (1, 4), {'length': 5, 'crossings': 1}), ((0, 4), (0, 5), {'length': 5, 'crossings': 1}), ((1, 0), (2, 0), {'length': 2, 'crossings': 1}), ((1, 0), (1, 1), {'length': 2, 'crossings': 1}), ((1, 1), (2, 1), {'length': 3, 'crossings': 2}), ((1, 1), (1, 2), {'length': 3, 'crossings': 2}), ((1, 2), (2, 2), {'length': 4, 'crossings': 2}), ((1, 

In [31]:
def pareto_dominates(a, b):
    """Check if the vector in b Pareto dominates vector a.

    Note: The original code has been modified to work for our minimization problem.

    Args:
        a (ndarray): A numpy array.
        b (ndarray): A numpy array.

    Returns:
        bool: Whether vector b dominates vector a.
    """
    a = np.array(a)
    b = np.array(b)
    return np.all(a <= b) and np.any(a < b)


def p_prune(candidates):
    """Create a Pareto coverage set from a set of candidate points.

    References:
        .. [1] Roijers, D. M., & Whiteson, S. (2017). Multi-objective decision making. 34, 129–129.
            https://doi.org/10.2200/S00765ED1V01Y201704AIM034

    Args:
        candidates (Set[Tuple]): A set of vectors.

    Returns:
        Set[Tuple]: A Pareto coverage set.
    """
    pcs = set()
    while candidates:
        vector = candidates.pop()

        for alternative in candidates:
            if pareto_dominates(alternative, vector):
                vector = alternative

        to_remove = set(vector)
        for alternative in candidates:
            if pareto_dominates(vector, alternative):
                to_remove.add(alternative)

        candidates -= to_remove
        pcs.add(vector)
    return pcs


def pvi(G, T, objectives, max_iter=1000):
    start = time.time()
    nd_vectors = [[{tuple(np.full(2, np.inf))} for _ in range(len(G.nodes))] for _ in range(len(G.nodes))] # Initialisation of nodes
    is_last = False
    converged = False
    count = 0

    for n, current_node in enumerate(G.nodes):
        if current_node == T:  # We've reached the terminal state
            nd_vectors[n] = [{(0, 0)} for _ in G.nodes]
            # converged = True
            break

    # nd_vectors_update = copy.deepcopy(nd_vectors)

    for run in range(max_iter):  # We execute the algorithm for a number of iterations.
        print(f'Value Iteration number: {run}')

    # while '{(inf, inf)}' in str(nd_vectors_update):
    # converged = False
    # while not converged:  # Run until convergence
    #     converged = True

        for n, current_node in enumerate(G.nodes):  # Loop over all states. Note: current_node is an object; n=number
            if current_node == T:
                break

            for nk, neighbor in enumerate(G.nodes): #Note: neighbor is an object; k=number
                if neighbor not in G.neighbors(current_node):
                    continue

                edge = G[current_node][neighbor]

                cost = []
                for i in objectives:
                  cost.append(edge[i])

                cost = np.array(cost)
                results = set()

                for v_list in nd_vectors[nk]:
                    for value_vec in v_list:
                        results.add(tuple(cost+value_vec)) # The set of candidate vectors

                results = p_prune(results)
                # print(f"result:{results}")

                nd_vectors[n][nk] = results

                # if nd_vectors[n][nk] != results:
                #     converged = False

        if converged:
            break

        nd_vectors = copy.deepcopy(nd_vectors)  # Else perform a deep copy and go again.
        # count+=1
        # print(f"count:{count}")


    end = time.time()
    elapsed_seconds = (end - start)
    print("Seconds elapsed: " + str(elapsed_seconds))

    return nd_vectors


objectives = ('length', 'crossings')

pvi_result = pvi(G, T, objectives)
print(pvi_result)


Value Iteration number: 0
Value Iteration number: 1
Value Iteration number: 2
Value Iteration number: 3
Value Iteration number: 4
Value Iteration number: 5
Value Iteration number: 6
Value Iteration number: 7
Value Iteration number: 8
Value Iteration number: 9
Value Iteration number: 10
Value Iteration number: 11
Value Iteration number: 12
Value Iteration number: 13
Value Iteration number: 14
Value Iteration number: 15
Value Iteration number: 16
Value Iteration number: 17
Value Iteration number: 18
Value Iteration number: 19
Value Iteration number: 20
Value Iteration number: 21
Value Iteration number: 22
Value Iteration number: 23
Value Iteration number: 24
Value Iteration number: 25
Value Iteration number: 26
Value Iteration number: 27
Value Iteration number: 28
Value Iteration number: 29
Value Iteration number: 30
Value Iteration number: 31
Value Iteration number: 32
Value Iteration number: 33
Value Iteration number: 34
Value Iteration number: 35
Value Iteration number: 36
Value Itera

In [4]:
map_amsterdam = gpd.read_file("Sidewalk_width_crossings_smaller random.geojson") #Read in the map with radius 250m and ~1000 nodes

# Objectives
objective1 = map_amsterdam['length']
objective2 = map_amsterdam['crossing']
objective3 = map_amsterdam['obstacle_free_width']

objectives = ('length', 'crossing')

In [11]:
import random
random_number = random.randint(1, 10)
for i in map_amsterdam['crossing']:
    map_amsterdam['crossing'][i] = random.randint(1, 10)
map_amsterdam['crossing']

/var/folders/kt/0pld6_md7nqfv750n335g64r0000gn/T/ipykernel_1598/779318583.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_amsterdam['crossing'][i] = random.randint(1, 10)


0      6.0
1      9.0
2      8.0
3      0.0
4      9.0
      ... 
225    1.0
226    1.0
227    1.0
228    1.0
229    1.0
Name: crossing, Length: 230, dtype: float64

In [14]:
# "0.9-1.8m": 0, "1.8-2.9m": 0, "<0.9m": 0, ">2.9m": 0
objective4 = map_amsterdam['0.9-1.8m'] * map_amsterdam['length']
objective4


0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
225    0.0
226    0.0
227    0.0
228    0.0
229    0.0
Length: 230, dtype: float64

In [3]:
# Create a NetworkX graph from the map
G = momepy.gdf_to_nx(map_amsterdam, approach='primal')
nodes = G.nodes
edges = G.edges


/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [8]:
# S = (122245.37633330293, 486126.8581684635) #very first node
# T = (122253.09793657108, 486219.18429932056)
S = (122245.37633330293, 486126.8581684635) #very first node
T = (122320.31466476223, 486327.5294561802)

In [9]:
values = []
for edge in G.edges:
    length = G.edges[edge]["length"]
    crossing = G.edges[edge]["crossing"]
    values.append([length, crossing])


In [10]:
pareto_front = paretoset(values)


In [11]:
pareto_paths = []
for i in pareto_front:
    edge = list(G.edges)[i]  # Get the edge from the index
    path = nx.shortest_path(G, source=S, target=T, method='dijkstra')  # Find the shortest path
    pareto_paths.append(path)

print(pareto_paths)

[[(122245.37633330293, 486126.8581684635), (122254.86602688645, 486129.80052856216), (122264.35426393585, 486132.74411788705), (122273.84081337851, 486135.69101398275), (122283.19823912054, 486138.9678026403), (122284.1816391946, 486139.19731384714), (122290.12705889999, 486148.4950394649), (122296.3898333376, 486151.2069021054), (122293.35529070727, 486160.70100324217), (122290.30564961619, 486170.1963395323), (122289.24584929895, 486173.25169782597), (122318.78284010534, 486193.54636295186), (122312.999838772, 486201.0746268706), (122309.41870472197, 486210.39752307127), (122305.90796557168, 486219.74544990633), (122302.50516522641, 486229.13200873986), (122299.20022373537, 486238.5538360078), (122295.99837830248, 486248.0130868562), (122292.87489444291, 486257.5008015807), (122262.21512810217, 486251.1722638347), (122258.88243648612, 486260.58853595395), (122283.81709680714, 486281.1675979253), (122282.66333889941, 486284.36990515806), (122279.27332236386, 486293.76013232773), (1222

/var/folders/kt/0pld6_md7nqfv750n335g64r0000gn/T/ipykernel_2659/212776972.py:3: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  edge = list(G.edges)[i]  # Get the edge from the index
